In [1]:
from data_utils.data_loader import load_data
from data_utils import preprocess
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import numpy as np
N_CLASSES = 174

In [2]:
data, label_dict = load_data("./data/something-something-mini")

In [3]:
videos, labels = preprocess.extract_videos_and_labels(data['train'], n_classes=N_CLASSES)
frames, labels = preprocess.extract_frames_and_labels(data['train'], n_classes=N_CLASSES)

In [4]:
try:
    from data_utils.metadata_loader import MetadataLoader
    from data_utils.video_loader import VideoLoader
except ModuleNotFoundError:
    from metadata_loader import MetadataLoader
    from video_loader import VideoLoader

label_folder_path = './data/something-something-mini-anno'

# train_file_path = "{}/train_videofolder.txt".format(label_folder_path)
# valid_file_path = "{}/val_videofolder.txt".format(label_folder_path)
# test_file_path = "{}/test_videofolder.txt".format(label_folder_path)


root_path = "./data/something-something-mini"
frame_path = "{}-frame".format(root_path)
anno_path = "{}-anno".format(root_path)



metadata_loader = MetadataLoader(label_folder_path=anno_path)
metadata = metadata_loader.load_metadata()




## 1) Dataset of paths to the video folders

In [33]:
frame_path = "./data/something-something-mini-frame/*"

list_ds = tf.data.Dataset.list_files(frame_path) # dataset of paths
for filename in list_ds.take(5):
    print(filename.numpy())

b'./data/something-something-mini-frame/5'
b'./data/something-something-mini-frame/9'
b'./data/something-something-mini-frame/1'
b'./data/something-something-mini-frame/3'
b'./data/something-something-mini-frame/7'


## 2) Dataset of datases of paths
(dataset has one dataset of paths to the video frames for each video)

In [6]:
# creates a dataset for each video
def dataset_from_path(file):
    return tf.data.Dataset.list_files(file+"/*",shuffle=False)

In [7]:
AUTOTUNE = tf.data.experimental.AUTOTUNE # makes the things faster

# a dataset of dataset of paths
list_ds_of_ds = list_ds.map(dataset_from_path, num_parallel_calls=AUTOTUNE)

## 3) Dataset  of loaded and stacked farmes

In [18]:
#### HERE I ASSUME THAT ALL FOLDERS IN ...mini-frame/* are part of the training set.
#--------------dont like this workaround with a------------------

list_of_label = [0]*11 # list of labels placed accordding to id
for i in metadata['train']:
    list_of_label[i] = metadata['train'][i]['action_label']
list_of_label = tf.convert_to_tensor(list_of_label)





def process_path(file_path):
      
    def _get_label(file_path):
        # convert the path to a list of path components
        parts = tf.strings.split(file_path, sep = "/")
        # The second to last is the class-directory
        k = tf.strings.to_number(parts[-2], out_type=tf.dtypes.int32)
        return list_of_label[k]
    
    img_width=455; img_height=256
    def _decode_image(frame): # from einar's script
        frame = tf.image.decode_jpeg(frame, channels=3)
        frame = tf.image.convert_image_dtype(frame, tf.float32)
        return tf.image.resize(frame, [img_width, img_height])
    
    #------------------------------------------------
    label = _get_label(file_path)
    img = tf.io.read_file(file_path) # from einar's script
    img = _decode_image(img)
    return img, label

In [19]:
def stack_images_from_path(ds):
    labeled_ds = ds.map(process_path, num_parallel_calls=AUTOTUNE)
    
    # temp variables
    label = tf.constant(0, dtype=tf.int32) 
    i = tf.constant(0)    
    imgs_combined = tf.TensorArray(dtype=tf.float32,size=1,dynamic_size=True,clear_after_read=False)
    
    for im,labels in labeled_ds:
        imgs_combined = imgs_combined.write(i, im)
        label = labels
        i = tf.add(i, 1)

    return imgs_combined.stack(), label

In [32]:
labeled_stacked_im_ds = list_ds_of_ds.map(stack_images_from_path, num_parallel_calls=AUTOTUNE)
for stacked_img, label in labeled_stacked_im_ds:
    print("shape:",stacked_img.shape, "label:",label.numpy())

shape: (36, 455, 256, 3) label: 129
shape: (42, 455, 256, 3) label: 13
shape: (36, 455, 256, 3) label: 104
shape: (48, 455, 256, 3) label: 68
shape: (24, 455, 256, 3) label: 43
shape: (57, 455, 256, 3) label: 0
shape: (43, 455, 256, 3) label: 75
shape: (31, 455, 256, 3) label: 94
shape: (44, 455, 256, 3) label: 6
shape: (47, 455, 256, 3) label: 114


## 4) Pad the stacked frames so that all videos have the same size( batches possible)

In [26]:
NR = 70
def pad(stacked_im, label):
          
    #nr = NR - tf.constant(stacked_im.get_shape.as_list()[0])
    nr = NR -stacked_im.get_shape().as_list()[0]
    
    paddings = tf.constant([[0, nr], [0, 0], [0,0], [0,0]])
    new = tf.pad(stacked_im,paddings,"CONSTANT")
    return new, label

# wrap it in tf.py_function to run it eagerly
def pad_fn(stacked_im, label):
    padded_im, label = tf.py_function(pad, 
                                       inp=[stacked_im, label], 
                                       Tout=(tf.float32, tf.int32))
    return padded_im, label


In [30]:
# the final dataset. 
labeled_padedim_ds = labeled_stacked_im_ds.map(pad_fn)

for paded_stacked_img, label in labeled_padedim_ds:
    print("shape:",paded_stacked_img.shape, "label:",label.numpy())

shape: (70, 455, 256, 3) label: 68
shape: (70, 455, 256, 3) label: 6
shape: (70, 455, 256, 3) label: 75
shape: (70, 455, 256, 3) label: 104
shape: (70, 455, 256, 3) label: 13
shape: (70, 455, 256, 3) label: 94
shape: (70, 455, 256, 3) label: 129
shape: (70, 455, 256, 3) label: 114
shape: (70, 455, 256, 3) label: 43
shape: (70, 455, 256, 3) label: 0
